In [12]:

!pip install ultralytics


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/914.6 kB ? eta -:--:--
   ----------- ---------------------------- 256.0/914.6 kB 7.9 MB/s eta 0:00:01
   -------------------------------- ------- 747.5/914.6 kB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 914.6/914.6 kB 8.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   --------------- ------------------------ 0.6/1.6 MB 12.9 MB/s eta 0:00:01
   ------------------------- -------------- 1.0/1.6 MB 10.5 MB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.6 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 8.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.6/204.1 MB 11.8 MB/s eta 0:00:18
   ---------------------------------------- 0.9/204.1 MB 10.0 MB/s eta 0:00:21
   ---

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Download the YOLOv8 model


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from tensorflow.keras.models import load_model

# Load YOLO model (use a pre-trained model or your custom one)
yolo_model = YOLO("yolov8n.pt")  

# Load seatbelt classifier model
classifier_model = load_model("seatbelt_noseatbelt_model.h5")

# Class labels and colors
results = {0: 'Seatbelt', 1: 'No Seatbelt'}
GR_dict = {0: (0, 255, 0), 1: (0, 0, 255)}  # Green = Seatbelt, Red = No Seatbelt

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    detections = yolo_model.predict(frame)  # Explicitly use .predict()

    # Extract detection results
    for r in detections:
        for box in r.boxes:  # Access bounding boxes correctly
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box
            conf = float(box.conf[0])  # Confidence
            cls = int(box.cls[0])  # Class ID

            # YOLO class 0 = 'person'
            if cls == 0 and conf > 0.5:
                # Extract upper body region (cropping half height)
                y_mid = int((y2 - y1) / 2)
                upper_body = frame[y1:y1 + y_mid, x1:x2]

                # Preprocess for seatbelt classifier
                resized_body = cv2.resize(upper_body, (200, 200))
                normalized_body = resized_body / 255.0
                reshaped_body = np.reshape(normalized_body, (1, 200, 200, 3))

                # Predict seatbelt status
                result = classifier_model.predict(reshaped_body)
                label = np.argmax(result, axis=1)[0]

                # Draw bounding box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), GR_dict[label], 2)
                cv2.rectangle(frame, (x1, y1 - 40), (x2, y1), GR_dict[label], -1)
                cv2.putText(frame, results[label], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Show frame
    cv2.imshow("YOLO Seatbelt Detection", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 266.3ms
Speed: 15.6ms preprocess, 266.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

0: 480x640 1 person, 156.6ms
Speed: 6.2ms preprocess, 156.6ms inference, 6.1ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step

0: 480x640 1 person, 108.7ms
Speed: 2.4ms preprocess, 108.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step

0: 480x640 1 person, 102.9ms
Speed: 0.0ms preprocess, 102.9ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step

0: 480x640 1 person, 110.5ms
Speed: 2.0ms preprocess, 110.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step

0: 480x640 1 person, 114.4ms
Speed: 0.0ms preprocess, 114.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step

0: 4